### **6.14 - Get notified with SLAs**

Qué pasa si quieres ser advertido si tus tareas están tardando más de lo esperado en completarse, por ejemplo, eso podría ser útil si tienes una tarea de larga ejecución, que definitivamente no quieres volver a ejecutar o tienes múltiples tareas y quieres asegurarte de que se completan en un período de tiempo determinado. Bueno, para eso no vas a usar timeouts, pero en su lugar, vas a usar otro mecanismo llamado "SLAs". Vamos a descubrirlos. **`¿Cuál es la diferencia entre SLAs y un timeout? Bueno, lo bueno de los SLAs es que siempre que defines SLA a una tarea, estás diciendo que, si tu tarea está tardando más de lo esperado en completarse, entonces sólo quieres recibir una notificación. No quieres detener tu tarea, como con el tiempo de espera (timeout). Y de nuevo, esto es extremadamente útil cuando tienes tareas de larga duración o algunas tareas que sólo quieres asegurarte de que se ejecutan en un determinado período de tiempo`**. Si no, entonces quieres echarles un vistazo. Ese es el objetivo de SLA. SLA es verificar que tu tarea se completa en un periodo de tiempo determinado. Ahora, con el fin de definir un SLA en Airflow, es bastante fácil, sólo tiene que definir el argumento. Así que vamos a hacer eso para la tarea "extract", por ejemplo, por lo que sólo tienes que definir "sla" igual a un objeto timedelta. Así, por ejemplo, puedes decir, quiero ser advertido si la tarea está tomando más de 5 minutos y eso es todo.

<center><img src="https://i.postimg.cc/9X9jTpT8/a1484.png"></center>

Pero tienes que tener mucho cuidado aquí, porque, el SLA no es relativo al "start time" de la tarea sino a la "execution date" de tu DAG, lo que significa, que si, por ejemplo, tienes una tarea justo antes de esta tarea

<center><img src="https://i.postimg.cc/qMdHG3xk/a1485.png"></center>

que tarda más de cinco minutos en completarse, entonces la tarea para la que se define el SLA, en ese caso aquí con cinco minutos, esa tendrá el SLA perdido.

<center><img src="https://i.postimg.cc/rFCXZwf5/a1486.png"></center>

Recibirás una notificación diciendo que esta tarea ha perdido su SLA, porque, es relativo a la "DAG execution date" y no al "task start time" donde está definido el SLA. Así, por ejemplo, si deseas verificar que tu DAG se completa en 10 minutos, sólo necesitas definir para la última tarea de tu DAG 

<center><img src="https://i.postimg.cc/YCrc5BPZ/a1487.png"></center>

el SLA de 10 minutos, porque, eso significa que todas tus tareas para tu DAG deben ser completadas dentro de los 10 minutos ya que el SLA es relativo al DAG execution date. Así que ten en cuenta esto porque es extremadamente confuso. Ahora, lo único que necesitas saber es que tan pronto como el SLA se pierda, el callback será llamado, y este callback es "sla_miss_callback" igual a una función Python, así que digamos, "_sla_miss_callback". 

<center><img src="https://i.postimg.cc/y8LqBr4d/a1488.png"></center>

**`Y lo importante a tener en cuenta aquí es que este callback se define a nivel DAG y no a nivel de tarea, lo que significa que esta función será llamada por todas tus tareas. Cualquier tarea que ha perdido su SLA, entonces este callback será llamado. No puedes tener diferentes callbacks para diferentes tareas, por eso es importante tener esto en cuenta`**. Entonces vamos a crear esa función, así que escribe "def _sla_miss_callback( )", y aquí tienes que añadir algunos argumentos:

<center><img src="https://i.postimg.cc/k5G05Fqy/a1489.png"></center>

Permítanme dar una explicación rápida aquí, porque esos parámetros no son tan de uso. Así que primero, ¿qué son "**`blocking_tis`**" y "**`blocking_task_list`**", porque, en realidad son más o menos los mismos parámetros. Bien, digamos que tienes dos tareas, T1 y T2. T2 depende de T1, y para ambas tareas, tienes SLA definido. Ahora bien, si T1 ha incumplido su SLA y T2 ha incumplido su SLA también, debido a T1, entonces T1 se considerará como una tarea de bloqueo (blocking task) para T2. Y ahí es cuando tienes a T1 como una instancia de tarea de bloqueo (blocking task instance). Ok, porque T1 perdió SLA, entonces T2 también ha perdido su SLA, por eso T1 es una tarea de bloqueo. Ahora, "**`slas`**" y "**`task_list`**" son más o menos los mismos parámetros, sólo que aquí sólo tienes los SLAs, por lo que todas las tareas que han perdido sus SLAs, y aquí tienes los correspondientes objetos de instancia de tarea. Por último, pero no menos importante, el objeto "**`dag`**" es bastante útil, si desea obtener alguna información acerca de su DAG. No dudes en, ya sabes, imprimir los diferentes parámetros que te darán mucha información. Así que, por ejemplo, vamos a imprimir:

<center><img src="https://i.postimg.cc/63GFrW72/a1490.png"></center>

Ahora, hay una cosa más que necesitas saber sobre los SLAs. **`Si disparas tu DAG manualmente, entonces los SLAs no se comprobarán`**. De nuevo, si disparas tu DAG manualmente y, por lo tanto, tu DAG no es disparado por el scheduler, entonces tus SLAs no serán comprobados. Y, por último, pero no menos importante, si quieres recibir la notificación tienes que configurar tu servidor SMTP, así como el parámetro de correo electrónico para tu DAG.  Si no lo haces, no recibirás ninguna notificación por correo electrónico y no hay otra forma de ser avisado cuando se incumplan los SLA. Esto es todo lo que necesitas saber sobre los SLAs. Son bastante difíciles de usar. Es posible que termines con algunos comportamientos extraños, pero aun así es bueno saber cómo funcionan y pueden ser extremadamente útiles para ti si tienes tareas de larga duración o tareas muy críticas para las que deseas ser advertido tan pronto como haya un problema.